# Conformational search using MD for multiple molecules
If we have several molecules we can use the multiple molecule conformational search. It generates forcefields and then uses the OpenMM MD driver with a centroid force in a high temperature MD run. A selected number of conformations are minimized.

In [ ]:
import veloxchem as vlx

Load your molecules and create a list containing those molecules.

In [ ]:
mol1 = vlx.Molecule.read_smiles('C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O')
mol2 = vlx.Molecule.read_smiles('O')
mol3 = vlx.Molecule.read_smiles('O')

molecules = [mol1, mol2, mol3]

Initiate and run the conformational search for multiple molecules.

In [ ]:
omm = vlx.OpenMMDynamics()
conformer_dict = omm.conformational_sampling_multiple(molecules, temperature = 800, nsteps = 100000, snapshots = 40, lowest_conformations = 5)

Show the structures

In [ ]:
for mol, e in zip(conformer_dict['molecules'], conformer_dict['energies']):
    mol.show()
    print(e,'kJ/mol')

Optimize the lowest conformers and performa a single point energy correction

In [ ]:
# Set up the functionals and basis sets used in the calculation
opt_functional = "BLYP"
opt_basis = "def2-svp"
sp_functional = "m06-l"
sp_basis = "def2-tzvp"

# Maximum number of optimization steps 
max_opt_steps = 15

energies = []
geometries = []
for i in range(len(conformer_dict['molecules'])):
    basis = vlx.MolecularBasis.read(conformer_dict['molecules'][i], opt_basis)
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.xcfun = opt_functional
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-3
    scf_drv.grid_level = 2
    #scf_drv.solvation_model = "cpcm"
    scf_drv.ri_coulomb = True
    results = scf_drv.compute(conformer_dict['molecules'][i], basis)

    opt_drv = vlx.OptimizationDriver(scf_drv)
    opt_drv.max_iter = max_opt_steps
    opt_drv.conv_maxiter = True
    opt_results = opt_drv.compute(conformer_dict['molecules'][i], basis, results)
    geometries.append(opt_results['final_geometry'])
    
    molecule = vlx.Molecule.read_xyz_string(opt_results['final_geometry'])
    basis = vlx.MolecularBasis.read(molecule, sp_basis)
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.xcfun = sp_functional
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-3
    scf_drv.grid_level = 4
    #scf_drv.solvation_model = "cpcm"
    scf_drv.ri_coulomb = True
    results = scf_drv.compute(molecule, basis)

    energies.append(results['scf_energy'])


In [ ]:
print(energies)

In [ ]:
mol = vlx.Molecule.read_xyz_string(geometries[0])
mol.show()